In [1]:
# Instalação de bibliotecas no Jupyter Notebook
!pip install requests pandas openpyxl python-dotenv


In [2]:
from dotenv import load_dotenv  # Para carregar o arquivo .env
import os  # Para acessar variáveis do sistema
import requests  # Para fazer as requisições à API
import pandas as pd  # Para manipular os dados e salvar na planilha


In [3]:
# Carregar o arquivo .env
load_dotenv()

# Obter a chave de API do Google Places
API_KEY = os.getenv("API_KEY")

# Verificar se a chave foi carregada corretamente
if not API_KEY:
    raise ValueError("A chave de API não foi encontrada. Verifique o arquivo .env.")


In [4]:
# URLs da API do Google Places
search_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
details_url = "https://maps.googleapis.com/maps/api/place/details/json"

# Parâmetros da busca
params = {
    "query": "clínicas médicas no Rio de Janeiro",  # Altere para o tipo de negócio e localização desejados
    "key": API_KEY
}


In [5]:
# Fazendo a requisição de busca
response = requests.get(search_url, params=params)
data = response.json()

# Lista para armazenar os dados dos negócios
negocios = []

# Limite de resultados
limite = 10
contador = 0

if "results" in data:
    for place in data["results"]:
        if contador >= limite:
            break  # Para o loop após atingir o limite

        # Coleta informações básicas
        place_id = place.get("place_id")
        name = place.get("name", "N/A")
        address = place.get("formatted_address", "N/A")
        rating = place.get("rating", "N/A")

        # Requisição para Place Details para obter telefone e site
        details_params = {
            "place_id": place_id,
            "fields": "formatted_phone_number,website",
            "key": API_KEY
        }
        details_response = requests.get(details_url, params=details_params)
        details_data = details_response.json()

        # Coletando telefone e site
        phone = details_data.get("result", {}).get("formatted_phone_number", "N/A")
        website = details_data.get("result", {}).get("website", "N/A")

        # Adicionando os dados à lista
        negocios.append({
            "Nome": name,
            "Telefone": phone,
            "Site/Instagram": website,
            "Endereço": address,
            "Avaliação": rating
        })

        contador += 1  # Incrementa o contador

else:
    print("Nenhum resultado encontrado.")


In [6]:
print(f"Status da requisição: {response.status_code}")
print("Resposta da API:")
print(data)


Status da requisição: 200
Resposta da API:
{'html_attributions': [], 'next_page_token': 'AVzFdblBnllgDdzD_NgCrsBMgoyOmxW3SYYfyLXpSvusywni2Y30_LI9U9XdXEVeAevXnf0TL5cZ5vA_tzHav_Wvcqb7vG7dFjLtrgIELBjo3c7b2K9g7UbqC5ArqydQzSNRdM8LMDLQzxQnnhfyb8yfajZgg22RxBhgX5Gqcs6Nzf7FbAaxRjfwMbZ75N4mnNX1o0p7J-9sMEwaYBQOloyRasEhFj5lLraAnC8aBRWlvBohawKukX5RQaW-19863sZUf9R7JGMzhmABRmn58HbnLl_kV3r1P8aUaN6jJ1jn1Ayw843rvXhr0LbO_6P8gh7C4-O7nJ697PiRhgU4nvVrSWbEMKFjMe18IswvPpZcAVsrWdgB4Si_uj0X9bVTejc_eFZqgshG5jepdATFfLuuunJd8NkGzkQm8oMLOjz9szeMpA', 'results': [{'business_status': 'OPERATIONAL', 'formatted_address': 'Av. Dom Hélder Câmara, 6548 - Pilares, Rio de Janeiro - RJ, 20771-002, Brazil', 'geometry': {'location': {'lat': -22.8822536, 'lng': -43.2924995}, 'viewport': {'northeast': {'lat': -22.88093367010728, 'lng': -43.29122107010727}, 'southwest': {'lat': -22.88363332989272, 'lng': -43.29392072989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png', 'icon_back

In [7]:
# Verifica se há dados coletados
if negocios:
    # Convertendo a lista para um DataFrame do pandas
    df = pd.DataFrame(negocios)

    # Salvando os dados em uma planilha Excel
    df.to_excel("negocios_locais.xlsx", index=False)
    print(f"Planilha 'negocios_locais.xlsx' salva com {len(negocios)} resultados!")
else:
    print("Nenhum dado para salvar.")


Planilha 'negocios_locais.xlsx' salva com 10 resultados!


In [ ]:
print(f"Status da requisição: {response.status_code}")
print("Resposta da API:")
print(data)


Status da requisição: 200
Resposta da API:
{'html_attributions': [], 'next_page_token': 'AVzFdblBnllgDdzD_NgCrsBMgoyOmxW3SYYfyLXpSvusywni2Y30_LI9U9XdXEVeAevXnf0TL5cZ5vA_tzHav_Wvcqb7vG7dFjLtrgIELBjo3c7b2K9g7UbqC5ArqydQzSNRdM8LMDLQzxQnnhfyb8yfajZgg22RxBhgX5Gqcs6Nzf7FbAaxRjfwMbZ75N4mnNX1o0p7J-9sMEwaYBQOloyRasEhFj5lLraAnC8aBRWlvBohawKukX5RQaW-19863sZUf9R7JGMzhmABRmn58HbnLl_kV3r1P8aUaN6jJ1jn1Ayw843rvXhr0LbO_6P8gh7C4-O7nJ697PiRhgU4nvVrSWbEMKFjMe18IswvPpZcAVsrWdgB4Si_uj0X9bVTejc_eFZqgshG5jepdATFfLuuunJd8NkGzkQm8oMLOjz9szeMpA', 'results': [{'business_status': 'OPERATIONAL', 'formatted_address': 'Av. Dom Hélder Câmara, 6548 - Pilares, Rio de Janeiro - RJ, 20771-002, Brazil', 'geometry': {'location': {'lat': -22.8822536, 'lng': -43.2924995}, 'viewport': {'northeast': {'lat': -22.88093367010728, 'lng': -43.29122107010727}, 'southwest': {'lat': -22.88363332989272, 'lng': -43.29392072989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png', 'icon_back